<a href="https://colab.research.google.com/github/fboldt/aulasann/blob/main/aula12a_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4971k      0  0:00:16  0:00:16 --:--:-- 9419k
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [ ]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"
for category in ["pos", "neg"]:
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [ ]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model="textattack/bert-base-uncased-imdb")

In [ ]:
results = pipe(["This is a great movie!", "It was a waste of time."])
for result in results:
  print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: LABEL_1, with score: 0.9666
label: LABEL_0, with score: 0.9911


In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import time

y_pred = []
test_labels = []
start = time.time()
for batch, (inputs, targets) in enumerate(test_ds):
    for i in range(len(inputs)):
        yp = pipe(inputs[i].numpy().decode('UTF-8')[:512])
        test_labels.append(int(targets[i].numpy()==1))
        y_pred.append(int(yp[0]['label']=='LABEL_1'))
    print(batch, time.time()-start, 's - ', 'acc:', accuracy_score(test_labels, y_pred))


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


0 1.319046974182129 s -  acc: 0.84375
1 5.092804908752441 s -  acc: 0.875
2 5.456376552581787 s -  acc: 0.8645833333333334
3 5.8120880126953125 s -  acc: 0.890625
4 6.273775339126587 s -  acc: 0.9
5 6.641906023025513 s -  acc: 0.8958333333333334
6 7.016528844833374 s -  acc: 0.9017857142857143
7 7.35978102684021 s -  acc: 0.8984375
8 7.709500789642334 s -  acc: 0.9027777777777778
9 8.058573007583618 s -  acc: 0.90625
10 8.404036521911621 s -  acc: 0.90625
11 8.749642610549927 s -  acc: 0.9036458333333334
12 9.099187135696411 s -  acc: 0.9014423076923077
13 9.426240682601929 s -  acc: 0.9084821428571429
14 9.757202386856079 s -  acc: 0.9125
15 10.11059308052063 s -  acc: 0.91015625
16 10.457001209259033 s -  acc: 0.90625
17 10.824322938919067 s -  acc: 0.9114583333333334
18 11.190278053283691 s -  acc: 0.9111842105263158
19 11.535745620727539 s -  acc: 0.9109375
20 11.864925622940063 s -  acc: 0.9092261904761905
21 12.291125535964966 s -  acc: 0.9090909090909091
22 12.70224905014038 s -